In [ ]:
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib

import sys
sys.path.insert(0, '../../')
import DLDMD as dl
import LossDLDMD as lf
import Data as dat
import Training as tr

%matplotlib inline

In [ ]:
def edmd(x, num_pred):
    x = tf.transpose(x, perm=[0, 2, 1])
    x_m = x[:, :, :-1]
    x_p = x[:, :, 1:]
    
    S, U, V = tf.linalg.svd(x_m, compute_uv=True, full_matrices=False)
    sm = np.max(S)
    r = S.shape[-1]
    Sri = tf.linalg.diag(1./S[:, :r])
    Ur = U[:, :, :r]
    Urh = tf.linalg.adjoint(Ur)
    Vr = V[:, :, :r]
    
    kmat = x_p @ Vr @ Sri @ Urh
    evals, evecs = tf.linalg.eig(kmat)
    phim = tf.linalg.solve(evecs, tf.cast(x_m, dtype=tf.complex128))
    x0 = phim[:, :, 0]
    x0 = x0[:, :, tf.newaxis]
    
    pred = tf.TensorArray(tf.complex128, size=num_pred)
    pred = pred.write(0, evecs @ x0)
    evals_iter = tf.identity(evals)
    for ii in range(num_pred):
        tmp = evecs @ tf.linalg.diag(evals_iter) @ x0
        pred = pred.write(ii, tmp)
        evals_iter = evals_iter * evals
    pred = tf.transpose(tf.squeeze(pred.stack()), perm=[1, 2, 0])
    return phim, evals, evecs, pred

# Setup

In [ ]:
# Figure parameters
plot_save_path = './analysis_results/'
font = {'family': 'DejaVu Sans', 'size': 18}
matplotlib.rc('font', **font)
fontsize = 18
figsize = (15, 10)
dpisave = 300

# Initialize the compute device
DEVICE = '/GPU:0'
GPUS = tf.config.experimental.list_physical_devices('GPU')
if GPUS:
    try:
        for gpu in GPUS:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
else:
    DEVICE = '/CPU:0'
    
tf.keras.backend.set_floatx('float64')  # !! Set precision for the entire model here
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))
print("Num GPUs available: {}".format(len(GPUS)))
print("Training at precision: {}".format(tf.keras.backend.floatx()))
print("Training on device: {}".format(DEVICE))

# Load model and data

In [ ]:
# SET THIS PATH (w/o file extension). Both '.pkl' and '.h5' files should have same name
model_path = './trained_models/pendulum_2021-10-05-0004/epoch_10_loss_-1.5'
hyp_params_path = model_path + '.pkl'
weight_path = model_path + '.h5'

# Load the hyper parameters
hyp_params = pickle.load(open(hyp_params_path, 'rb'))

# Set Tensorflow backend precision
tf.keras.backend.set_floatx(hyp_params['precision'])
print("Using precision: {}\n".format(tf.keras.backend.floatx()))

# Load evenly spaced rings for test trajectories
test_data = pickle.load(open('evenly_spaced_trajs.pkl', 'rb'))
test_data = tf.cast(test_data, dtype=hyp_params['precision'])
print("Test data shape: {}".format(test_data.shape))

In [ ]:
# Fix hyper parameters for running the model on test data
hyp_params['pretrain'] = False
hyp_params['batch_size'] = test_data.shape[0]
hyp_params['num_time_steps'] = test_data.shape[1]
hyp_params['latent_dim'] = test_data.shape[2]
hyp_params['phys_dim'] = test_data.shape[2]

# Load the trained DLDMD model weights
model = dl.DLDMD(hyp_params)
model.num_pred_steps = model.num_time_steps
model.time_final = int(model.num_time_steps*model.delta_t)
model(test_data)
model.load_weights(weight_path)

# Initialize the loss function
loss = lf.LossDLDMD(hyp_params)
print("Number of prediction steps: ", model.num_pred_steps)

# Run the DLDMD model

In [ ]:
with tf.device(DEVICE):
    [y, x_ae, x_adv, y_adv, weights, evals, evecs, phi] = model(test_data, training=False)
    losses = loss([y, x_ae, x_adv, y_adv, weights, evals, evecs, phi], test_data)

print("Loss: {loss:2.7f}".format(loss=losses.numpy()))
print("Log10 Loss: {loss:2.7f}".format(loss=np.log10(losses.numpy())))

# Run standard DMD

In [ ]:
# EDMD on the unencoded data
[phim, evals, evecs, x_pred] = edmd(test_data, num_pred=test_data.shape[1])
x_pred = np.real(tf.transpose(x_pred, perm=[0, 2, 1]))

# Visualize results

In [ ]:
fs = 20
ts = 20
lw = 2.0
ms = 20.0
figsize = (12, 12)
skip = 1

# DLDMD reconstruction
fig = plt.figure(1, figsize=figsize)
for ii in range(0, test_data.shape[0], skip):
    plt.plot(test_data[ii, :, 0], test_data[ii, :, 1], 'k', linestyle='solid', lw=lw)
    plt.plot(x_adv[ii, :, 0], x_adv[ii, :, 1], 'k', linestyle='dotted', ms=ms)
plt.plot(test_data[ii, :, 0], test_data[ii, :, 1], 'k', linestyle='solid', lw=lw, label='Test data')
plt.plot(x_adv[ii, 0, 0], x_adv[ii, 0, 1], 'k', linestyle='dotted', ms=20*ms, label='DLDMD')
plt.xlabel(r'$x$', fontsize=fs)
plt.ylabel(r'$\dot{x}$', fontsize=fs)
plt.legend(fontsize=fs, loc='upper right')
plt.axis('equal')
ax = plt.gca()
ax.tick_params(axis='both', which='major', labelsize=ts)
ax.tick_params(axis='both', which='minor', labelsize=ts)

# DMD reconstruction
fig = plt.figure(2, figsize=figsize)
for ii in range(0, test_data.shape[0], skip):
    plt.plot(test_data[ii, :, 0], test_data[ii, :, 1], 'k', linestyle='solid', lw=lw)
    plt.plot(x_pred[ii, :, 0], x_pred[ii, :, 1], 'k', linestyle='dotted', ms=ms)
plt.plot(test_data[ii, :, 0], test_data[ii, :, 1], 'k', linestyle='solid', lw=lw, label='Test data')
plt.plot(x_pred[ii, 0, 0], x_pred[ii, 0, 1], 'k', linestyle='dotted', ms=20*ms, label='DMD')
plt.xlabel(r'$x$', fontsize=fs)
plt.ylabel(r'$\dot{x}$', fontsize=fs)
plt.legend(fontsize=fs)
plt.axis('equal')
ax = plt.gca()
ax.tick_params(axis='both', which='major', labelsize=ts)
ax.tick_params(axis='both', which='minor', labelsize=ts)

# Plot the trajectories in phase space and latent space
fig = plt.figure(3, figsize=figsize)
for ii in range(0, test_data.shape[0], skip):
    plt.plot(test_data[ii, :, 0], test_data[ii, :, 1], 'k', linestyle='solid', lw=lw)
plt.xlabel(r'$x$', fontsize=fs)
plt.ylabel(r'$\dot{x}$', fontsize=fs)
plt.axis('equal')
ax = plt.gca()
ax.tick_params(axis='both', which='major', labelsize=ts)
ax.tick_params(axis='both', which='minor', labelsize=ts)

# Plot the trajectories in latent space
fig = plt.figure(4, figsize=figsize)
for ii in range(y_adv.shape[0]):
    plt.plot(y[ii, :, 0], y[ii, :, 1], 'k', linestyle='solid', ms=ms)
plt.xlabel(r'$\tilde{x}_{1}$', fontsize=fs)
plt.ylabel(r'$\tilde{x}_{2}$', fontsize=fs)
plt.axis('equal')
ax = plt.gca()
ax.tick_params(axis='both', which='major', labelsize=ts)
ax.tick_params(axis='both', which='minor', labelsize=ts)

plt.show()